In [ ]:
import os
os.getcwd()
os.chdir("../../..")
os.getcwd()

In [ ]:
from irec.data.implemented_datasets import *
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import argparse
from irec.model.psge.psge import PSGE
from irec.experiments.svd_dataset import load_svd
from irec.experiments.plot.matplotlib_utils.setup_plot import setup_plot

In [ ]:
setup_plot(162, fig_ratio=0.7, style_sheet="base", font_size=8)

In [ ]:
best_param = {
    "MovieLens1M": {
        "alpha": 0.4,
        "beta": 0.3,
    },
    "Amazon Electronics": {
        "alpha": 0.5,
        "beta": 0.3,
    },
    "Gowalla": {
        "alpha": 0.4,
        "beta": 0.3,
    },
}

In [ ]:
KCORE = 10
METRIC = "NDCG"
CUTOFF = 20
m_k = METRIC + "@" + str(CUTOFF)

In [ ]:
dataset = Movielens1M()
dataset.k_core = 10

In [ ]:
model = "PSGE"
preprocessed_dir = os.path.join(dataset.data_directory, "preprocessed")
k_core_dir = os.path.join(preprocessed_dir, f"k_core_{KCORE}")
save_path = os.path.join(k_core_dir, "psge_results")
save_name = "{}.csv".format(eval(model).name)
final_path = os.path.join(save_path, save_name)
res_df = pd.read_csv(final_path, index_col=0)
res_df["alg"] = eval(model).name

In [ ]:
alpha = best_param[dataset.name]["alpha"] 
beta = best_param[dataset.name]["beta"]
_, vt, s = load_svd(
                dataset, merge_val=True, alpha=alpha, beta=beta
            )

In [ ]:
# search for best k
best_perf = res_df[m_k].max()
best_k = res_df[res_df[m_k] == best_perf]["k"].values[0]

In [ ]:
best_perf

In [ ]:
from matplotlib.ticker import FormatStrFormatter
#ax = sns.lineplot(x="k", y=m_k, data=res_df, color="black")
ax = plt.gca()
plt.plot(res_df["k"].values, res_df[m_k].values, color="black", label="NDCG@20")
ax_handles, ax_label = ax.get_legend_handles_labels()
ax.set_ylabel("NDCG@20")
#ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

plt.axvline(x=best_k, ymax=1, ymin=0, ls=(0, (5,1)), alpha=1)

plt.xlabel("$k$")
ax2 = ax.twinx()
ax2.tick_params(direction="in", length=1.5, width=0.5)
perf_k = [0, *res_df["k"].values]
ax2.plot(perf_k, s[list(map(int,perf_k))], color="red", linestyle="--", label="$\lambda$")
ax2.set_ylabel("$\lambda$")
ax2.grid(False)
ax2_handles, ax2_label = ax2.get_legend_handles_labels()

handles, labels = [*ax_handles, *ax2_handles], [*ax_label, *ax2_label] 

plt.legend(handles, labels, bbox_to_anchor=(1,0.4), loc="upper right")
plt.gca().legend_.get_frame().set_linewidth(0.5)

plt.tight_layout(pad=0.5)
plt.savefig("{}/egcn_plot/{}.pdf".format(os.environ["HOME"], f"{dataset.name}_sp_perf"),)